In [4]:
import re
from math import log

def tokenize(text):
    file = open(text, "r", encoding="ISO-8859-1")
    lines = file.readlines()
    tokens = []
    extra_stuff = ''
    for i in range(len(lines)):
        lines[i] = lines[i].lower()
        x = lines[i].split(' ')
        x = x[:-1]
        x = [re.sub(r"\d+", "", z) for z in x]
        x = [re.sub(r"[,.?:;\"@#$%^&*()_|/+=<>\-\!\[\]]", "", z) for z in x]
        if extra_stuff in x:
            x.remove(extra_stuff)
        x = ['<s>']+x+['</s>']
        tokens.append(x)
#     print(tokens)
    return tokens

def bigram_dict(data):
    bigram_counts = {}
    indi_counts = {}
    for sentence in data:
        for letter_idx in range(len(sentence)-1):
            letter1 = sentence[letter_idx]
            letter2 = sentence[letter_idx + 1]
            if letter1 not in bigram_counts:
                bigram_counts[letter1] = {}
                bigram_counts[letter1][letter2] = 1
            else:
                if letter2 in bigram_counts[letter1]:
                    bigram_counts[letter1][letter2] += 1
                else:
                    bigram_counts[letter1][letter2] = 1
            if letter1 not in indi_counts:
                indi_counts[letter1] = 1
            else:
                indi_counts[letter1] += 1
    return indi_counts, bigram_counts

def bigram_prob_gts(indi_counts, bigram_counts, letter, prevletter, N, N_c):
    if prevletter in bigram_counts:
        if letter in bigram_counts[prevletter]:
            if (bigram_counts[prevletter][letter] + 1) not in N_c:
                num = bigram_counts[prevletter][letter]
                denom = indi_counts[prevletter]
            else:
                num = (bigram_counts[prevletter][letter]+1)*N_c[bigram_counts[prevletter][letter]+1] + bigram_counts[prevletter][letter]
                denom = N_c[bigram_counts[prevletter][letter]] + indi_counts[prevletter]
        else:
            num = 1
            denom = indi_counts[prevletter]
    else:
        num = N_c[1]
        denom = N+len(indi_counts.keys())
    num = float(num)
    prob = log(num / denom)
    return prob

def accuracy():
    file_out = open("wordLangId.out", "r", encoding="ISO-8859-1")
    output_lines = file_out.readlines()
    file_sol = open("LangId.sol", "r", encoding="ISO-8859-1")
    sol_lines = file_sol.readlines()
    total = len(output_lines)
    count = 0
    for idx in range(len(output_lines)):
        if output_lines[idx] == sol_lines[idx]:
            count += 1
    acc = (count / total)*100
    return acc, count, total

def get_bigram_gts(indi_counts, bigram_counts):
    N_c = {}
    for l in bigram_counts:
        for pl in bigram_counts[l]:
            if bigram_counts[l][pl] in N_c:
                N_c[bigram_counts[l][pl]] += 1
            else:
                N_c[bigram_counts[l][pl]] = 1
    N = 0.0
    for i in N_c.keys():
        N += N_c[i]
#     print(N_c)
    return N, N_c
        
eng_tokens = tokenize("LangId.train.English")
french_tokens = tokenize("LangId.train.French")
it_tokens = tokenize("LangId.train.Italian")
eng_ic, eng_bc = bigram_dict(eng_tokens)
french_ic, french_bc = bigram_dict(french_tokens)
it_ic, it_bc = bigram_dict(it_tokens)
test_tokens = tokenize("LangId.test")
with open('wordLangId2.out', 'w') as output:           
    N_eng, N_c_eng = get_bigram_gts(eng_ic, eng_bc)
    N_french, N_c_french = get_bigram_gts(french_ic, french_bc)
    N_it, N_c_it = get_bigram_gts(it_ic, it_bc)
    
    count = 1
    for sen in test_tokens:
        prob_eng = 0.0
        prob_fr = 0.0
        prob_it = 0.0
        
        for letter_idx in range(1, len(sen)):
            prob_eng += bigram_prob_gts(eng_ic, eng_bc, sen[letter_idx], sen[letter_idx-1], N_eng, N_c_eng)
            prob_fr += bigram_prob_gts(french_ic, french_bc, sen[letter_idx], sen[letter_idx-1], N_french, N_c_french)
            prob_it += bigram_prob_gts(it_ic, it_bc, sen[letter_idx], sen[letter_idx-1], N_it, N_c_it)
        if prob_eng >= prob_fr and prob_eng >= prob_it:
            lang = "English"
        elif prob_fr >= prob_eng and prob_fr >= prob_it:
            lang = "French"
        else:
            lang = "Italian"
        out_text = str(count)+" "+lang+"\n"
        output.write(out_text)
        count+=1
accuracy, count, total = accuracy()
print("Accuracy of the model is", accuracy)
print(count,"sentences match the solution out of", total)


Accuracy of the model is 98.33333333333333
295 sentences match the solution out of 300
